In [1]:
from pydantic import BaseModel

# Define your Pydantic model to structure the output
class ExpandedAnswer(BaseModel):
    initial_answer: str
    detailed_answer: str


In [2]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Define the prompt template
initial_prompt = PromptTemplate.from_template(
    template="Given the context: '{context}', what is the answer to the question: '{question}'?"
)

# Initialize the LLM
# llm = ChatOpenAI(model="gpt-4", temperature=0.7)
llm = ChatOpenAI(
                model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
                base_url="http://10.2.125.37:1234/v1",
                api_key="lm-studio"
            )


In [3]:
# Chain using the pipe syntax and structured output with Pydantic
chain = initial_prompt | llm.with_structured_output(ExpandedAnswer)

In [4]:
# Define the input data
context = "AI is increasingly being adopted in various industries, including healthcare..."
question = "What are the key benefits of AI in healthcare?"

# Run the chain with the input data
response = chain.invoke({
    "context": context,
    "question": question
})

# Access the structured output
print(response.initial_answer)
print(response.detailed_answer)

AttributeError: 'NoneType' object has no attribute 'initial_answer'

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel

# Define your Pydantic model
class ExpandedAnswer(BaseModel):
    initial_answer: str
    detailed_answer: str

# Define the prompt template
initial_prompt = PromptTemplate.from_template(
    template="Given the context: '{context}', what is the answer to the question: '{question}'?"
)

# Initialize the LLM with your custom configuration
llm = ChatOpenAI(
    model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
    base_url="http://10.2.125.37:1234/v1",
    api_key="lm-studio"
)

# Use chaining with structured output
chain = initial_prompt | llm.with_structured_output(ExpandedAnswer)

# Define the input data
context = "AI is increasingly being adopted in various industries, including healthcare..."
question = "What are the key benefits of AI in healthcare?"

# Run the chain with the input data
response = chain.invoke({
    "context": context,
    "question": question
})

# Debug: Print the raw response
print("Raw response:", response)

# Safely access the structured output
if response:
    print(response.initial_answer)
    print(response.detailed_answer)
else:
    print("The model did not return a valid response.")
